In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/traveling-santa-2018-prime-paths/sample_submission.csv
/kaggle/input/traveling-santa-2018-prime-paths/cities.csv


In [2]:
import os
import pandas as pd
cities = pd.read_csv('../input/traveling-santa-2018-prime-paths/cities.csv', index_col=['CityId'], nrows=None)
cities = cities * 1000  # not sure if coords are rounded as concorde
cities.head()

,X,Y
CityId,,
0,3.168367e+05,2.202341e+06
1,4.377406e+06,3.366021e+05
2,3.454158e+06,2.820053e+06
3,4.688099e+06,2.935898e+06
4,1.010697e+06,3.236751e+06


In [3]:
%%bash -e
wget http://akira.ruc.dk/~keld/research/LKH/LKH-2.0.9.tgz
tar xvfz LKH-2.0.9.tgz
cd LKH-2.0.9
make

LKH-2.0.9/
LKH-2.0.9/pr2392.par
LKH-2.0.9/E3k.0.par
LKH-2.0.9/Makefile
LKH-2.0.9/xray14012_1.tsp
LKH-2.0.9/LKH-2.0.9.tgz
LKH-2.0.9/E3k.0.pi
LKH-2.0.9/E3k.0.tsp
LKH-2.0.9/pr2392.tsp
LKH-2.0.9/DOC/
LKH-2.0.9/README.txt
LKH-2.0.9/xray14012_1.par
LKH-2.0.9/SRC/
LKH-2.0.9/SRC/RestoreTour.c
LKH-2.0.9/SRC/SolveKMeansSubproblems.c
LKH-2.0.9/SRC/IsCommonEdge.c
LKH-2.0.9/SRC/ReadProblem.c
LKH-2.0.9/SRC/BestKOptMove.c
LKH-2.0.9/SRC/Distance_SPECIAL.c
LKH-2.0.9/SRC/CreateCandidateSet.c
LKH-2.0.9/SRC/OBJ/
LKH-2.0.9/SRC/Forbidden.c
LKH-2.0.9/SRC/Best5OptMove.c
LKH-2.0.9/SRC/RecordBetterTour.c
LKH-2.0.9/SRC/Best4OptMove.c
LKH-2.0.9/SRC/Exclude.c
LKH-2.0.9/SRC/C.c
LKH-2.0.9/SRC/IsCandidate.c
LKH-2.0.9/SRC/Make3OptMove.c
LKH-2.0.9/SRC/Make2OptMove.c
LKH-2.0.9/SRC/ResetCandidateSet.c
LKH-2.0.9/SRC/LKHmain.c
LKH-2.0.9/SRC/SolveSFCSubproblems.c
LKH-2.0.9/SRC/ERXT.c
LKH-2.0.9/SRC/fscanint.c
LKH-2.0.9/SRC/eprintf.c
LKH-2.0.9/SRC/Gain23.c
LKH-2.0.9/SRC/Heap.c
LKH-2.0.9/SRC/GetTime.c
LKH-2.0.9/SRC/SolveRoheSu

--2021-12-28 16:33:21--  http://akira.ruc.dk/~keld/research/LKH/LKH-2.0.9.tgz
Resolving akira.ruc.dk (akira.ruc.dk)... 130.225.220.230
Connecting to akira.ruc.dk (akira.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1508124 (1.4M) [application/x-gzip]
Saving to: ‘LKH-2.0.9.tgz’

     0K .......... .......... .......... .......... ..........  3% 90.9K 16s
    50K .......... .......... .......... .......... ..........  6%  182K 11s
   100K .......... .......... .......... .......... .......... 10%  182K 10s
   150K .......... .......... .......... .......... .......... 13% 11.4M 7s
   200K .......... .......... .......... .......... .......... 16%  181K 7s
   250K .......... .......... .......... .......... .......... 20%  181K 6s
   300K .......... .......... .......... .......... .......... 23%  176K 6s
   350K .......... .......... .......... .......... .......... 27%  175K 6s
   400K .......... .......... .......... .......... .......

In [4]:
def write_tsp(nodes, filename, name='traveling-santa-2018-prime-paths'):
    # From https://www.kaggle.com/blacksix/concorde-for-5-hours.
    with open(filename, 'w') as f:
        f.write('NAME : %s\n' % name)
        f.write('COMMENT : %s\n' % name)
        f.write('TYPE : TSP\n')
        f.write('DIMENSION : %d\n' % len(cities))
        f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
        f.write('NODE_COORD_SECTION\n')
        for row in cities.itertuples():
            f.write('%d %.11f %.11f\n' % (row.Index + 1, row.X, row.Y))
        f.write('EOF\n')

write_tsp(cities, '../working/LKH-2.0.9/cities.tsp')

In [5]:
def write_parameters(parameters, filename='../working/LKH-2.0.9/params.par'):
    with open(filename, 'w') as f:
        for param, value in parameters:
            f.write("{} = {}\n".format(param, value))
    print("Parameters saved as", filename)

parameters = [
    ("PROBLEM_FILE", "cities.tsp"),
    ("OUTPUT_TOUR_FILE", "tsp_solution.csv"),
    ("SEED", 2018),
    ('CANDIDATE_SET_TYPE', 'POPMUSIC'), #'NEAREST-NEIGHBOR', 'ALPHA'),
    ('INITIAL_PERIOD', 10000),
    ('MAX_TRIALS', 1000),
]
write_parameters(parameters)

Parameters saved as ../working/LKH-2.0.9/params.par


In [6]:
%%bash -e
cd ./LKH-2.0.9
timeout 18000s ./LKH params.par

PARAMETER_FILE = params.par
Reading PROBLEM_FILE: "cities.tsp" ... done
ASCENT_CANDIDATES = 50
BACKBONE_TRIALS = 0
BACKTRACKING = NO
# CANDIDATE_FILE =
CANDIDATE_SET_TYPE = POPMUSIC
# EDGE_FILE =
EXCESS = 5.0564e-06
EXTRA_CANDIDATES = 0 
EXTRA_CANDIDATE_SET_TYPE = QUADRANT
GAIN23 = YES
GAIN_CRITERION = YES
INITIAL_PERIOD = 10000
INITIAL_STEP_SIZE = 1
INITIAL_TOUR_ALGORITHM = WALK
# INITIAL_TOUR_FILE = 
INITIAL_TOUR_FRACTION = 1.000
# INPUT_TOUR_FILE = 
KICK_TYPE = 0
KICKS = 1
# MAX_BREADTH =
MAX_CANDIDATES = 5 
MAX_SWAPS = 197769
MAX_TRIALS = 1000
# MERGE_TOUR_FILE =
MOVE_TYPE = 5
# NONSEQUENTIAL_MOVE_TYPE = 5
# OPTIMUM =
OUTPUT_TOUR_FILE = tsp_solution.csv
PATCHING_A = 1 
PATCHING_C = 0 
# PI_FILE = 
POPMUSIC_INITIAL_TOUR = NO
POPMUSIC_MAX_NEIGHBORS = 5
POPMUSIC_SAMPLE_SIZE = 10
POPMUSIC_SOLUTIONS = 50
POPMUSIC_TRIALS = 1
# POPULATION_SIZE = 0
PRECISION = 100
PROBLEM_FILE = cities.tsp
RECOMBINATION = IPT
RESTRICTED_SEARCH = YES
RUNS = 10
SEED = 2018
STOP_AT_OPTIMUM = YES
SUBGRADIENT =

CalledProcessError: Command 'b'cd ./LKH-2.0.9\ntimeout 18000s ./LKH params.par\n'' returned non-zero exit status 124.

In [ ]:
def read_tour(filename):
    tour = []
    for line in open(filename).readlines():
        line = line.replace('\n', '')
        try:
            tour.append(int(line) - 1)
        except ValueError as e:
            pass  # skip if not a city id (int)
    return tour[:-1]

tour = read_tour('../working/LKH-2.0.9/tsp_solution.csv')
print("Tour length", len(tour))

In [ ]:
import numpy as np
import sympy

def score_tour(tour):
    df = cities.reindex(tour + [0]).reset_index()
    primes = list(sympy.primerange(0, len(cities)))
    df['prime'] = df.CityId.isin(primes).astype(int)
    df['dist'] = np.hypot(df.X - df.X.shift(-1), df.Y - df.Y.shift(-1))
    df['penalty'] = df['dist'][9::10] * (1 - df['prime'][9::10]) * 0.1
    return df.dist.sum() + df.penalty.sum()

def write_submission(tour, filename):
    assert set(tour) == set(range(len(tour)))
    pd.DataFrame({'Path': list(tour) + [0]}).to_csv(filename, index=False)

print("Final score", score_tour(tour))
write_submission(tour, 'submission.csv')